In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import urllib
import urllib.parse as urlp
import io
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

def get_time_series(start_date,end_date,latitude,longitude,variable):
    """
    Calls the data rods service to get a time series
    """
    base_url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi"
    query_parameters = {
        "variable": variable,
        "type": "asc2",
        "location": f"GEOM:POINT({longitude}, {latitude})",
        "startDate": start_date,
        "endDate": end_date,
    }
    full_url = base_url+"?"+ \
         "&".join(["{}={}".format(key,urlp.quote(query_parameters[key])) for key in query_parameters])
    print(full_url)
    iteration = 0
    done = False
    while not done and iteration < 5:
        r=requests.get(full_url)
        if r.status_code == 200:
            done = True
        else:
            iteration +=1
    
    if not done:
        raise Exception(f"Error code {r.status_code} from url {full_url} : {r.text}")
    
    return r.text

def parse_time_series(ts_str):
    """
    Parses the response from data rods.
    """
    lines = ts_str.split("\n")
    parameters = {}
    for line in lines[2:11]:
        key,value = line.split("=")
        parameters[key] = value
    
    
    df = pd.read_table(io.StringIO(ts_str),sep="\t",
                       names=["time","data"],
                       header=10,parse_dates=["time"])
    return parameters, df

#43.674901654747046, -79.53730583391066

In [2]:
def createData(lat,long):
    df_ts = parse_time_series(
            get_time_series(
                start_date="2012-06-01T00", 
                end_date="2022-05-31T23",
                latitude=lat,
                longitude=long,
                variable="GLDAS2:GLDAS_NOAH025_3H_v2.1:Rainf_f_tavg"))
    df_ts1 = parse_time_series(
            get_time_series(
                start_date="2000-01-01T00", 
                end_date="2025-06-27T23",
                latitude=lat,
                longitude=long,
                variable="GLDAS2:GLDAS_NOAH025_3H_v2.1:Rainf_tavg"))
    df_ts2 = parse_time_series(
            get_time_series(
                start_date="2000-01-01T00", 
                end_date="2025-06-27T23",
                latitude=lat,
                longitude=long,
                variable="GLDAS2:GLDAS_NOAH025_3H_v2.1:Qair_f_inst"))
    df_ts3 = parse_time_series(
            get_time_series(
                start_date="2000-01-01T00", 
                end_date="2025-06-27T23",
                latitude=lat,
                longitude=long,
                variable="GLDAS2:GLDAS_NOAH025_3H_v2.1:Psurf_f_inst"))
    df1 = df_ts1[1].rename({'data': 'rain average'},axis='columns')
    df2 = df_ts2[1].rename({'data': 'Humidity','time':'t2'},axis='columns')
    df3 = df_ts3[1].rename({'data': 'Pressure (Pa)','time':'t3'},axis='columns')
    combinedDF = pd.concat([df1,df2,df3], axis=1, join="inner")
    combinedDF.drop(columns=['t2', 't3'], inplace=True)
    combinedDF['Year'] = pd.to_datetime(combinedDF['time']).dt.year
    combinedDF['Month'] = pd.to_datetime(combinedDF['time']).dt.month
    combinedDF['Day'] = pd.to_datetime(combinedDF['time']).dt.day
    combinedDF['Hour'] = pd.to_datetime(combinedDF['time']).dt.hour
    combinedDF.drop(columns=['time'], inplace=True)
    combinedDF.isnull().sum()
    combinedDF['Longitude'] = long
    combinedDF['Latitude'] = lat
    monthly_avg_humidity = combinedDF.groupby('Month')['Humidity'].mean()
    monthly_avg_air_pressure = combinedDF.groupby('Month')['Pressure (Pa)'].mean()

    X = combinedDF.drop('rain average', axis=1)  
    y = combinedDF['rain average'].values



    return combinedDF

In [3]:
combineDF = createData(43.67, -79.54)
combineDF2 = createData(43.67+2,-79.54+2)
combineDF3 = createData(43.67-2,-79.54-2)

https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi?variable=GLDAS2%3AGLDAS_NOAH025_3H_v2.1%3ARainf_f_tavg&type=asc2&location=GEOM%3APOINT%28-79.54%2C%2043.67%29&startDate=2012-06-01T00&endDate=2022-05-31T23
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi?variable=GLDAS2%3AGLDAS_NOAH025_3H_v2.1%3ARainf_tavg&type=asc2&location=GEOM%3APOINT%28-79.54%2C%2043.67%29&startDate=2000-01-01T00&endDate=2025-06-27T23
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi?variable=GLDAS2%3AGLDAS_NOAH025_3H_v2.1%3AQair_f_inst&type=asc2&location=GEOM%3APOINT%28-79.54%2C%2043.67%29&startDate=2000-01-01T00&endDate=2025-06-27T23
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi?variable=GLDAS2%3AGLDAS_NOAH025_3H_v2.1%3APsurf_f_inst&type=asc2&location=GEOM%3APOINT%28-79.54%2C%2043.67%29&startDate=2000-01-01T00&endDate=2025-06-27T23
https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/access/timeseries.cgi?variable=GLDAS2%3AGLDAS_NOAH0

In [4]:
masterDF = pd.concat([combineDF,combineDF2,combineDF3], ignore_index=True)
masterDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223437 entries, 0 to 223436
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   rain average   223437 non-null  float64
 1   Humidity       223437 non-null  float64
 2   Pressure (Pa)  223437 non-null  float64
 3   Year           223437 non-null  int32  
 4   Month          223437 non-null  int32  
 5   Day            223437 non-null  int32  
 6   Hour           223437 non-null  int32  
 7   Longitude      223437 non-null  float64
 8   Latitude       223437 non-null  float64
dtypes: float64(5), int32(4)
memory usage: 11.9 MB


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

def createModel(masterDF):
    monthly_avg_humidity = masterDF.groupby('Month')['Humidity'].mean()
    monthly_avg_air_pressure = masterDF.groupby('Month')['Pressure (Pa)'].mean()

    X = masterDF.drop('rain average', axis=1)  
    y = masterDF['rain average'].values

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.3 , random_state = 89)
    
    linear_model = LinearRegression() 
    linear_model.fit(X_train, y_train)

    predictions = linear_model.predict(X_test)
    linear_model.score(X_test, predictions)
    print(linear_model.score(X_test, y_test)) #test it
    print(linear_model.coef_) #print significance of feature of linear model
    return linear_model

In [6]:
# create a sample prediction for our matrix
def samplePrediction(linear_model):
    sample = {'Humidity': [monthly_avg_humidity[6]],
            'Pressure (Pa)': [monthly_avg_air_pressure[6]],
            'Year': [2023],
            'Month': [6],
            'Day': [15],
            'Hour': [12],
            'Longitude': [-79.59],
            'Latitude': [43.80],
    }
    sample_df = pd.DataFrame(sample)
    linear_model.predict(sample_df)

def displayData(combinedDF):
    # display number of precipitation events above a certain threshold
    threshold = 3.96e-05  # define threshold for heavy precipitation
    heavy_precip_events = combinedDF[combinedDF['rain average'] > threshold]
    print(f"Number of heavy precipitation events: {len(heavy_precip_events)}")
    print("number of total events:", len(combinedDF))

In [11]:
import pickle

def createPickle(linear_model):
    with open('rain_model_three_locations.pkl','wb') as f:
        pickle.dump(linear_model, f)
        f.close()


In [12]:
linear_model = createModel(masterDF)
createPickle(linear_model)

0.8344900666450409
[ 8.01080040e+04  1.45818458e+00  2.34065747e+01 -3.34836130e+01
 -7.20965885e-02  2.58072070e-01  1.49203260e+03  1.49203260e+03]
